In [ ]:
import sys
import os
import matplotlib.pyplot as plt

# Add the src directory to the path. TEMPORARY FIX
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../..")))

from models.unet.unet import UNet
from models.unet.unet_formes import UNetFormes
from src.data_processing.dataset_loader import CoastData

In [ ]:
# Load the data to split it and save it to a dict
path = os.path.abspath(os.path.join(os.getcwd(), "../../data/processed/"))
data = CoastData(data_path=path, name="arenaldentem")

data_split = data.split_data()

In [ ]:
unet = UNet(num_classes=3, experiment_name="test_experiments", use_mlflow=True)

In [ ]:
# Load the data to the model
data = unet.load_data(data_split, UNetFormes)

In [ ]:
artifact_path = os.path.abspath(os.path.join(path, "../../artifacts/"))

# Train the model
unet.train(epochs=2, artifact_path=artifact_path)